In [1]:
%sql

CREATE TABLE STREAMING_DATA(
RECORD_ID INTEGER,
A_NUMBER DOUBLE,
A_TEXT STRING
)
LOCATION '/dbfs/mnt/test_data/streaming_db/streaming_data'

In [2]:
%sh

ls /dbfs/dbfs/mnt/test_data/streaming_db/streaming_data

In [3]:
%sql

INSERT INTO TABLE STREAMING_DATA VALUES (1, 1.0, 'first_record');

In [4]:
%sql

SELECT * FROM STREAMING_DATA

RECORD_ID,A_NUMBER,A_TEXT
1,1.0,first_record


In [5]:
%sh

ls /dbfs/dbfs/mnt/test_data/streaming_db/streaming_data

_committed_2894894926104193824
part-00000-tid-2894894926104193824-a22f6dea-6de0-454c-a7f2-f9035ad3c506-0-1-c000
_started_2894894926104193824
_SUCCESS